Metadata for browser:
- doi: url
- title: Expanded Name (i.e., Review and Herald (1.1) Dec. 12, 1870)
- authors: blank
- journal: expand name
- volume: number
- issue: number
- date: yyyy-mm-dd
- page: page

In [1]:
import csv
import datetime
import GoH.utilities
import pandas as pd
import re
import urllib

In [2]:
base_url = "http://documents.adventistarchives.org/Periodicals/"

In [3]:
title_keys = {"ADV": "Training School Advocate",
"AmSn": "American Sentinel",
"ARAI": "Advent Review and Sabbath Herald Anniversary Issue",
"CE": "Christian Education",
"CUV": "Welcome Visitor (Columbia Union Visitor)",
"EDU": "Christian Educator",
"GCB": "General Conference Bulletin",
"GH": "Gospel Herald",
"GOH": "Gospel of Health",
"GS": "Gospel Sickle",
"HM": "Home Missionary",
"HR": "Health Reformer",
"IR": "Indiana Reporter",
"LB": "Life Boat",
"LH": "Life and Health",
"LibM": "Liberty",
"LUH": "Lake Union Herald",
"NMN": "North Michigan News Sheet",
"PHJ": "Pacific Health Journal and Temperance Advocate",
"PTAR": "Present Truth (Advent Review)",
"PUR": "Pacific Union Recorder",
"RH": "Review and Herald",
"Sligo": "Sligonian",
"SOL": "Sentinel of Liberty",
"ST": "Signs of the Times",
"SUW": "Report of Progress, Southern Union Conference",
"TCOG": "The Church Officer's Gazette",
"TMM": "The Missionary Magazine",
"WMH": "West Michigan Herald",
"YI": "Youth's Instructor"}

In [4]:
df = pd.read_table('../../data/target_300_10_Sample.txt',
                  header=None,
                  names=['doc_id', 'label', 'content'])

In [5]:
df[:10]

,doc_id,label,content
0,ADV18981201-V02-01-page12.txt,en,uncommon thing member boarding school form cl...
1,ADV18981201-V02-01-page13.txt,en,lex truly lord able save uttermost bound long...
2,ADV18981201-V02-01-page15.txt,en,personal experience lord impressed written te...
3,ADV18981201-V02-01-page16.txt,en,pride heart thought foolish think thing isa p...
4,ADV18981201-V02-01-page20.txt,en,anew important department college college con...
5,ADV18990101-V01-01-page34.txt,en,advocate church school miss writes school jun...
6,ADV18990101-V01-01-page35.txt,en,the_advocate younger year old frightened thre...
7,ADV18990101-V01-01-page44.txt,en,the_advocate library reading_room furnished m...
8,ADV18990101-V01-01-page45.txt,en,the_advocate student desire work order reduce...
9,ADV18990201-V01-02-page54.txt,en,advocate weight pound ounce silver charger cu...


In [6]:
def get_split_id(doc_id):
    return doc_id.split('-')

def construct_url(base_url, abrev, split_id):
    return urllib.parse.urljoin(base_url, "{}/{}.pdf".format(abrev, "-".join(split_id[:-1])) )

def get_date(split_id):  
    return re.search(r'[0-9]+', split_id[0]).group()

def get_volume(split_id):   
    return re.search(r'[0-9]+', split_id[1]).group()
    
def get_issue(split_id):
    return split_id[2]

def get_page(split_id):
    return re.search(r'[0-9]+', split_id[3]).group()   

In [19]:
with open('../../browser/data/meta.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    for index, row in df.iterrows():
        _id = row['doc_id'].strip()
#         print(row['doc_id'])
        abrev = GoH.utilities.get_title(_id)
        split_id = get_split_id(_id)
        date_data = get_date(split_id)
        url = construct_url(base_url, abrev, split_id)
        
        try:
            if len(date_data) == 4:
                date = datetime.datetime.strptime(get_date(split_id), "%Y")           
            elif len(date_data) < 7:
                date = datetime.datetime.strptime(get_date(split_id), "%Y%m")
            else:
                date = datetime.datetime.strptime(get_date(split_id), "%Y%m%d")
        except:
            date = datetime.datetime.strptime(get_date(split_id)[:6], "%Y%m")

        try:
            volume = get_volume(split_id)
        except:
            volume = "XX"
            
        try:
            issue = get_issue(split_id)
        except:
            issue = "XX"
            
        page = get_page(split_id)
        journal = title_keys[abrev]
        title = "{} (Vol. {}.{}) {}, page {}".format(journal, volume, issue, date.strftime('%b %d, %Y'), page)
        writer.writerow([_id, title, "NA", journal, volume, issue, date.isoformat(), page])